In [ ]:
import sys, os
import random
import numpy as np
import pandas as pd 

import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split

In [ ]:
DATA_DIR = "../input/jane-street-market-prediction"

In [ ]:
# read all the files
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
features_df = pd.read_csv(os.path.join(DATA_DIR, "features.csv"))
example_test = pd.read_csv(os.path.join(DATA_DIR, "example_test.csv"))
sample_prediction_df = pd.read_csv(os.path.join(DATA_DIR, "example_sample_submission.csv"))

In [ ]:
train_features = [c for c in train_df.columns if 'feature' in c]

# fill with -999
train_df.fillna(-999, inplace=True)
# add 'action' to the training data
train_df['action'] = (train_df['resp'] > 0).astype('int')

train, val = train_test_split(train_df, test_size=0.2)

In [ ]:
class JSMPDataset(Dataset):
  def __init__(self, features, targets, test=False):
    self.features = features
    self.targets = targets
  
  def __len__(self):
    return len(self.features)
  
  def __getitem__(self, idx):
    return torch.Tensor(self.features[idx]), torch.LongTensor(self.targets[idx]).squeeze()

In [ ]:
class JSMPDataModule(pl.LightningDataModule):
  def __init__(self, train_df, val_df, features_col, targets_col):
    super().__init__()
    self.train_df = train_df
    self.val_df = val_df
    self.features_col = features_col
    self.targets_col = targets_col
    
  def prepare_data(self):
    pass
    
  def setup(self, stage=None):
    self.train_dataset = JSMPDataset(self.train_df[self.features_col].values, self.train_df[self.targets_col].values)
    self.val_dataset = JSMPDataset(self.val_df[self.features_col].values, self.val_df[self.targets_col].values)
    self.test_dataset = None
    
  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=4096, num_workers=4)
  
  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=4096, num_workers=4)
  
  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=4096, num_workers=4)

In [ ]:
class JSMPModule(pl.LightningModule):
  def __init__(self, num_features,
  num_classes,
  learning_rate,
  hidden_sizes,
  dropout):
    # define the model
    super().__init__()
    # required to save all the hyperparameters in this model
    self.save_hyperparameters()
    self.learning_rate = learning_rate
    layers = []
    layers.append(nn.Linear(num_features, hidden_sizes[0]))
    layers.append(nn.BatchNorm1d(hidden_sizes[0]))
    layers.append(nn.Dropout(dropout))
    for i, hidden_size in enumerate(hidden_sizes):
      layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
      layers.append(nn.BatchNorm1d(hidden_sizes[i]))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout))
    layers.append(nn.Linear(hidden_sizes[-1], num_classes))
    self.model = nn.Sequential(*layers)
  
  def forward(self, features):
    return self.model(features)
  
  def configure_optimizers(self):
    return torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
  
  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    return loss
  
  def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    metrics = {'val_loss': loss}
    self.log_dict(metrics)
    return metrics

In [ ]:
# checkpoint_callback = ModelCheckpoint(
#   monitor='val_loss',
#   # save in the folder that is of the same name as the file
#   dirpath=f'{__file__.split(".")[0]}/',
#   mode='min',
#   filename='nn-model-{epoch:02d}-{val_loss:.2f}',
# )

early_stopping_callback = EarlyStopping(
  monitor='val_loss',
  patience=3
)

In [ ]:
# train
dm = JSMPDataModule(train, val, train_features, ['action'])
jsmpmodel = JSMPModule(
  num_features = len(train_features),
  num_classes=2,
  learning_rate=1e-3,
  hidden_sizes=[400,800,800,400],
  dropout=0.2289
)
trainer = pl.Trainer(gpus=1,
  callbacks=[early_stopping_callback],
  max_epochs=200
)
trainer.fit(jsmpmodel, dm)

In [ ]:
# put the model in eval mode
jsmpmodel.eval()

# init env
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

In [ ]:
# make the predictions
for (test_df, predict_df) in iter_test:
    test_df.fillna(-999, inplace=True)
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    y_preds = jsmpmodel(torch.FloatTensor(X_test.values)).detach().numpy()
    predict_df.action = y_preds.argmax()
    env.predict(predict_df)